Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

In [ ]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [ ]:
!cd Tensorflow && git clone https://github.com/tensorflow/models
#git config --global http.version HTTP/1.1

In [ ]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

In [ ]:
#!!! Check this
# make dir like below
#copy pipline.config
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!copy {'Tensorflow\workspace\pre-trained-models\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\pipeline.config'} {'Tensorflow\workspace\models\my_ssd_mobnet'}

In [3]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [4]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)


2.10.1
1.23.0


In [ ]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
model_config = config['model']
detection_model = model_builder.build(model_config=model_config, is_training=True)

In [ ]:
# Access the feature extractor of the detection model
feature_extractor = detection_model._feature_extractor

# Check if the model is in training mode
print("Is training:", feature_extractor._is_training)

# Access the MobileNet base model name (if applicable)
# Note: For SSD MobileNet, there may not be an attribute directly named as base model. 
# You might need to explore the structure to find equivalent attributes.
print("Model name:", feature_extractor._name)

# Example of accessing backbone network (if applicable)
# SSD MobileNet feature extractor typically uses MobileNetV2 or similar as its backbone
print("Backbone network:", feature_extractor._base_model_initialized)

print(feature_extractor.classification_backbone)
# Example of getting the feature map generator
# SSD models often have methods to get specific feature layers or maps
feature_maps = feature_extractor._extract_features
print("Feature maps:", feature_maps)

# Access the conv definitions
conv_defs = feature_extractor._conv_defs
print("Conv defs:", conv_defs)

# Explore other attributes and methods as needed for your specific tasks


In [ ]:
import tensorflow as tf

# Load a pre-trained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

# Print the summary to see the layers
base_model.summary()

# Get the layers
layers = base_model.layers

# Print the total number of layers
print("Total number of layers:", len(layers))

# List layer names
for i, layer in enumerate(layers):
    print(f"Layer {i}: {layer.name}")


In [ ]:
# Access the feature extractor of the detection model
feature_extractor = detection_model._feature_extractor

# Print the layers attribute
print("Layers:", feature_extractor.layers)

# Print the submodules attribute
print("Submodules:", feature_extractor.submodules)

# Print the flattened layers
print("Flattened layers:", feature_extractor._flatten_layers())

# Print trainable variables
print("Trainable variables:", feature_extractor.trainable_variables)

# Print trainable weights
print("Trainable weights:", feature_extractor.trainable_weights)

# Print all variables
print("All variables:", feature_extractor.variables)

# Print all weights
print("All weights:", feature_extractor.weights)



In [ ]:
config

In [ ]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

In [ ]:
#tensorboard --logdir=train/

In [ ]:
import os
import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

PATH_TO_CFG = "Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config"
PATH_TO_CKPT = "Tensorflow/workspace/models/my_ssd_mobnet"

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

In [ ]:
PATH_TO_LABELS = 'Tensorflow/workspace/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
IMAGE_PATHS = 'Tensorflow/workspace/images/processed_images/zm1_2_img003_jpg.rf.7b72906c2bc1c235f283e570fa9dccab.jpg'

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

img = cv2.imread(IMAGE_PATHS)
image_np = np.array(img)

plt.imshow(cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils

warnings.filterwarnings('ignore')  # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):

    return np.array(Image.open(path))

IMAGE_PATHS = IMAGE_PATHS  # Update with your image path

print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image_np = load_image_into_numpy_array(IMAGE_PATHS)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

# Convert to numpy arrays and remove batch dimension
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

# Filter detections to keep only the class with ID 1
desired_class_id = 1
indices = np.where(detections['detection_classes'] == desired_class_id)[0]

filtered_boxes = detections['detection_boxes'][indices]
filtered_classes = detections['detection_classes'][indices]
filtered_scores = detections['detection_scores'][indices]

# Count the number of bounding boxes with confidence scores over 70%
confidence_threshold = 0.70
high_confidence_indices = np.where(filtered_scores > confidence_threshold)[0]
high_confidence_count = len(high_confidence_indices)

# Filter out detections below the confidence threshold
filtered_boxes = filtered_boxes[high_confidence_indices]
filtered_classes = filtered_classes[high_confidence_indices]
filtered_scores = filtered_scores[high_confidence_indices]

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    filtered_boxes,
    filtered_classes + label_id_offset,
    filtered_scores,
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=confidence_threshold,
    agnostic_mode=False
)

plt.figure()
plt.imshow(image_np_with_detections)
print('Done')
plt.show()

print(f'Number of bounding boxes with confidence over 70%: {high_confidence_count}')


In [10]:
import os
import tensorflow as tf
from tensorboard.backend.event_processing import event_accumulator

log_dir = "Tensorflow/workspace/models/my_ssd_mobnet/train"
event_acc = event_accumulator.EventAccumulator(log_dir)
event_acc.Reload()

# Check available tags
print("Available scalar tags:", event_acc.Tags()['scalars'])


# Extract loss and accuracy
loss_events = event_acc.Scalars('Loss')
accuracy_events = event_acc.Scalars('Accuracy')

loss_steps = [event.step for event in loss_events]
loss_values = [event.value for event in loss_events]

accuracy_steps = [event.step for event in accuracy_events]
accuracy_values = [event.value for event in accuracy_events]

import matplotlib.pyplot as plt

# Plot loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(loss_steps, loss_values, label='Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(accuracy_steps, accuracy_values, label='Accuracy')
plt.xlabel('Steps')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Available scalar tags: []


KeyError: 'Key Loss was not found in Reservoir'

In [11]:
import os
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

log_dir = "Tensorflow/workspace/models/my_ssd_mobnet/train"  # Replace with your actual logs directory
event_acc = EventAccumulator(log_dir)
event_acc.Reload()

# Check available tags
print("Available scalar tags:", event_acc.Tags()['scalars'])

# Extract and plot loss and accuracy
loss_events = event_acc.Scalars('Loss/total_loss')
accuracy_events = event_acc.Scalars('Metrics/accuracy')

# Example: Extract steps and values
loss_steps = [event.step for event in loss_events]
loss_values = [event.value for event in loss_events]

accuracy_steps = [event.step for event in accuracy_events]
accuracy_values = [event.value for event in accuracy_events]

import matplotlib.pyplot as plt

# Plot loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(loss_steps, loss_values, label='Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(accuracy_steps, accuracy_values, label='Accuracy')
plt.xlabel('Steps')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Available scalar tags: []


KeyError: 'Key Loss/total_loss was not found in Reservoir'